# Natural SQL 7B

In [ ]:
pip install transformers==4.35.2 accelerate sqlparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 28.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("chatdb/natural-sql-7b")
model = AutoModelForCausalLM.from_pretrained(
    "chatdb/natural-sql-7b",
    device_map="auto",
    torch_dtype=torch.float16,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
questions = ['Show me the day with the most users joining', 'Show me the project that has a task with the most comments', 'What is the ratio of users with gmail addresses vs without?', 'create a new table named work and add attriutes like work id  int64']

for question in questions:
    prompt = f"""
    ### Task

    Generate a SQL query to answer the following question: `{question}`

    ### PostgreSQL Database Schema
    The query will run on a database with the following schema:
    ```
    CREATE TABLE users (
        user_id SERIAL PRIMARY KEY,
        username VARCHAR(50) NOT NULL,
        email VARCHAR(100) NOT NULL,
        password_hash TEXT NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
    );

    CREATE TABLE projects (
        project_id SERIAL PRIMARY KEY,
        project_name VARCHAR(100) NOT NULL,
        description TEXT,
        start_date DATE,
        end_date DATE,
        owner_id INTEGER REFERENCES users(user_id)
    );

    CREATE TABLE tasks (
        task_id SERIAL PRIMARY KEY,
        task_name VARCHAR(100) NOT NULL,
        description TEXT,
        due_date DATE,
        status VARCHAR(50),
        project_id INTEGER REFERENCES projects(project_id)
    );

    CREATE TABLE taskassignments (
        assignment_id SERIAL PRIMARY KEY,
        task_id INTEGER REFERENCES tasks(task_id),
        user_id INTEGER REFERENCES users(user_id),
        assigned_date DATE NOT NULL DEFAULT CURRENT_TIMESTAMP
    );

    CREATE TABLE comments (
        comment_id SERIAL PRIMARY KEY,
        content TEXT NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
        task_id INTEGER REFERENCES tasks(task_id),
        user_id INTEGER REFERENCES users(user_id)
    );
    ```

    ### Answer
    Here is the SQL query that answers the question: `{question}`
    ```sql
    """

    print ("Question: " + question)
    print ("SQL: ")

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=100001,
        pad_token_id=100001,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,

    )

    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    print(outputs[0].split("```sql")[-1])


Question: Show me the day with the most users joining
SQL: 

     SELECT created_at::date AS join_date, COUNT(*) AS user_count
     FROM users
     GROUP BY join_date
     ORDER BY user_count DESC
     LIMIT 1;
Question: Show me the project that has a task with the most comments
SQL: 

     SELECT p.project_id, p.project_name, COUNT(c.comment_id) AS comment_count
     FROM projects p
     JOIN tasks t ON p.project_id = t.project_id
     JOIN comments c ON t.task_id = c.task_id
     GROUP BY p.project_id
     ORDER BY comment_count DESC
     LIMIT 1;
Question: What is the ratio of users with gmail addresses vs without?
SQL: 

     SELECT
        SUM(CASE WHEN email LIKE '%@gmail.com%' THEN 1 ELSE 0 END) AS gmail_users,
        SUM(CASE WHEN email NOT LIKE '%@gmail.com%' THEN 1 ELSE 0 END) AS non_gmail_users,
        (SUM(CASE WHEN email LIKE '%@gmail.com%' THEN 1 ELSE 0 END)::FLOAT / NULLIF(SUM(CASE WHEN email NOT LIKE '%@gmail.com%' THEN 1 ELSE 0 END), 0)) AS gmail_ratio
    FROM users